In [22]:
import stanza

In [23]:
nlp = stanza.Pipeline(lang='de', processors='tokenize,mwt,pos,lemma,depparse')

2025-11-18 12:27:06 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-11-18 12:27:07 INFO: Downloaded file to C:\Users\vivek\stanza_resources\resources.json
2025-11-18 12:27:08 INFO: Loading these models for language: de (German):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |

2025-11-18 12:27:08 INFO: Using device: cpu
2025-11-18 12:27:08 INFO: Loading: tokenize
2025-11-18 12:27:08 INFO: Loading: mwt
2025-11-18 12:27:08 INFO: Loading: pos
2025-11-18 12:27:09 INFO: Loading: lemma
2025-11-18 12:27:17 INFO: Loading: depparse
2025-11-18 12:27:17 INFO: Done loading processors!


In [24]:
def check_present(doc):
    for sent in doc.sentences:
        for w in sent.words:
            upos = w.upos or ""
            feats = (w.feats or "")

            if upos in ("VERB", "AUX") and feats:
                if "Tense=Pres" in feats:
                    return True

    return False


def check_perfect(doc):
    has_aux = False
    has_part = False
    for sent in doc.sentences:
        for w in sent.words:
            upos = w.upos or ""
            feats = (w.feats or "")
            lemma = getattr(w, "lemma", "") or ""
            xpos = getattr(w, "xpos", "") or ""

            if not has_aux:
                if upos in ("VERB", "AUX") and feats:
                    # finite auxiliary 'haben' or 'sein'
                    if (lemma.lower() in ("haben", "sein")):
                        # Prefer a finite auxiliary to avoid things like "... zu haben"
                        if ("VerbForm=Fin" in feats) or ("Mood=" in feats):
                            has_aux = True

            if "VerbForm=Part" in feats or ("VVPP" in xpos):
                has_part = True

            if has_aux and has_part:
                return True

    return False


def check_past(doc):
    for sent in doc.sentences:
        for w in sent.words:
            upos = w.upos or ""
            feats = (w.feats or "")

            if upos in ("VERB", "AUX") and feats:
                if "Tense=Past" in feats:
                    return True
    return False


def check_und_oder(doc):
    for sent in doc.sentences:
        for w in sent.words:
            text = w.text
            if text in ("und", "oder"):
                return True
    return False


def check_causal(doc):
    for sent in doc.sentences:
        for w in sent.words:
            text = w.text
            if text in ("weil", "da", "denn"):
                return True
    return False


def check_dass(doc):
    for sent in doc.sentences:
        for w in sent.words:
            text = w.text
            if text == "dass":
                return True
    return False


def check_wenn(doc):
    for sent in doc.sentences:
        for w in sent.words:
            text = w.text
            if text == "wann":
                return True
    return False


def check_attribute(doc):
    for sent in doc.sentences:
        for w in sent.words:
            feats = (w.feats or "")
            if "Case=Nom" in feats or "Case=Acc" in feats or "Case=Dat" in feats or "Case=Gen" in feats:
                return True
    return False


def check_adverb(doc):
    for sent in doc.sentences:
        for w in sent.words:
            deprel = w.deprel or ""
            if deprel == "obl":
                return True
    return False


def check_prepositional(doc):
    for sent in doc.sentences:
        for w in sent.words:
            deprel = w.deprel or ""
            if deprel == "case":
                return True
    return False


def check_impersonal(doc):
    for sent in doc.sentences:
        for w in sent.words:
            text = w.text
            deprel = w.deprel or ""
            if text in ("es", "man") and deprel in ("expl", "nsubj"):
                return True
    return False


def check_passive(doc):
    for sent in doc.sentences:
        for w in sent.words:
            feats = (w.feats or "")
            text = (w.text or "").lower()
            if "Voice=Pass" in feats or (text in ("wird", "wurden", "wurde") and any("VerbForm=Part" in (child.feats or "") for child in sent.words if child.head == w.id)):
                return True
    return False


def check_nominal(doc):
    for sent in doc.sentences:
        for w in sent.words:
            upos = w.upos or ""
            text = (w.text or "").lower()
            if upos == "NOUN" and any(text.endswith(s) for s in ("ung", "keit", "heit", "nis", "tum")):
                return True
    return False


def check_final(doc):
    for sent in doc.sentences:
        for w in sent.words:
            text = (w.text or "").lower()
            if text in ("damit", "um", "zu"):
                return True
    return False


def check_adversative(doc):
    for sent in doc.sentences:
        for w in sent.words:
            text = (w.text or "").lower()
            if text in ("obwohl", "während", "wohingegen"):
                return True
    return False


def check_direct(doc):
    for sent in doc.sentences:
        for w in sent.words:
            text = (w.text or "").lower()
            if text in ('„', "“"):
                return True
    return False

In [25]:
def analyze_text(text: str) -> dict:
    doc = nlp(text)
    out = {}
    out["präsens"] = check_present(doc)
    out["präteritum"] = check_past(doc)
    out["perfekt"] = check_perfect(doc)
    out["und_oder"] = check_und_oder(doc)
    out["kausale_nebensätze"] = check_causal(doc)
    out["nebensätze_dass"] = check_dass(doc)
    out["nebensätze_wenn"] = check_wenn(doc)
    out["attribute"] = check_attribute(doc)
    out["adverbialen"] = check_adverb(doc)
    out["präposition"] = check_prepositional(doc)
    out["unpersönliche"] = check_impersonal(doc)
    out["passivkonstruktionen"] = check_passive(doc)
    out["nominalisierungens"] = check_nominal(doc)
    out["nebensätze_finale"] = check_final(doc)
    out["ebensätze_adversative"] = check_adversative(doc)
    out["direkte_sprache"] = check_direct(doc)

    return out

In [26]:
# Lernziel 1: Hauptsatz im Präsens

example = "Der Hund läuft."
analyze_text(example)

{'präsens': True,
 'präteritum': False,
 'perfekt': False,
 'und_oder': False,
 'kausale_nebensätze': False,
 'nebensätze_dass': False,
 'nebensätze_wenn': False,
 'attribute': True,
 'adverbialen': False,
 'präposition': False,
 'unpersönliche': False,
 'passivkonstruktionen': False,
 'nominalisierungens': False,
 'nebensätze_finale': False,
 'ebensätze_adversative': False,
 'direkte_sprache': False}

In [27]:
# Lernziel 2: Hauptsatz im Perfekt

example = "Der Hund ist gelaufen."
analyze_text(example)

{'präsens': True,
 'präteritum': False,
 'perfekt': True,
 'und_oder': False,
 'kausale_nebensätze': False,
 'nebensätze_dass': False,
 'nebensätze_wenn': False,
 'attribute': True,
 'adverbialen': False,
 'präposition': False,
 'unpersönliche': False,
 'passivkonstruktionen': False,
 'nominalisierungens': False,
 'nebensätze_finale': False,
 'ebensätze_adversative': False,
 'direkte_sprache': False}

In [28]:
# Lernziel 3: Hauptsatz im Präteritum
example = "Der Hund lief."
analyze_text(example)

{'präsens': False,
 'präteritum': True,
 'perfekt': False,
 'und_oder': False,
 'kausale_nebensätze': False,
 'nebensätze_dass': False,
 'nebensätze_wenn': False,
 'attribute': True,
 'adverbialen': False,
 'präposition': False,
 'unpersönliche': False,
 'passivkonstruktionen': False,
 'nominalisierungens': False,
 'nebensätze_finale': False,
 'ebensätze_adversative': False,
 'direkte_sprache': False}

In [29]:
# Lernziel 4: Hauptsatzverbindungen mit und/oder

example = "Der Hund lief und schaute den Mann an."
analyze_text(example)

{'präsens': False,
 'präteritum': True,
 'perfekt': False,
 'und_oder': True,
 'kausale_nebensätze': False,
 'nebensätze_dass': False,
 'nebensätze_wenn': False,
 'attribute': True,
 'adverbialen': False,
 'präposition': False,
 'unpersönliche': False,
 'passivkonstruktionen': False,
 'nominalisierungens': False,
 'nebensätze_finale': False,
 'ebensätze_adversative': False,
 'direkte_sprache': False}

In [30]:
# Lernziel 5: Kausale Nebensätze

example = "Der Hund lief weg, weil er Angst hatte."
analyze_text(example)

{'präsens': False,
 'präteritum': True,
 'perfekt': False,
 'und_oder': False,
 'kausale_nebensätze': True,
 'nebensätze_dass': False,
 'nebensätze_wenn': False,
 'attribute': True,
 'adverbialen': False,
 'präposition': False,
 'unpersönliche': False,
 'passivkonstruktionen': False,
 'nominalisierungens': False,
 'nebensätze_finale': False,
 'ebensätze_adversative': False,
 'direkte_sprache': False}

In [31]:
# Lernziel 6: Nebensätze mit „dass“

example = "Der Hund hörte, dass der Mann nach ihm rief."
analyze_text(example)

{'präsens': False,
 'präteritum': True,
 'perfekt': False,
 'und_oder': False,
 'kausale_nebensätze': False,
 'nebensätze_dass': True,
 'nebensätze_wenn': False,
 'attribute': True,
 'adverbialen': True,
 'präposition': True,
 'unpersönliche': False,
 'passivkonstruktionen': False,
 'nominalisierungens': False,
 'nebensätze_finale': False,
 'ebensätze_adversative': False,
 'direkte_sprache': False}

In [32]:
# Lernziel 7: Nebensätze mit „wenn“

example = "Der Hund kommt immer zurück, wenn der Mann nach ihm ruft."
analyze_text(example)

{'präsens': True,
 'präteritum': False,
 'perfekt': False,
 'und_oder': False,
 'kausale_nebensätze': False,
 'nebensätze_dass': False,
 'nebensätze_wenn': False,
 'attribute': True,
 'adverbialen': True,
 'präposition': True,
 'unpersönliche': False,
 'passivkonstruktionen': False,
 'nominalisierungens': False,
 'nebensätze_finale': False,
 'ebensätze_adversative': False,
 'direkte_sprache': False}

In [33]:
# Lernziel 8: Attribute (Nominativ, Akkusativ, Dativ, Genitiv)

example = ""
analyze_text(example)

{'präsens': False,
 'präteritum': False,
 'perfekt': False,
 'und_oder': False,
 'kausale_nebensätze': False,
 'nebensätze_dass': False,
 'nebensätze_wenn': False,
 'attribute': False,
 'adverbialen': False,
 'präposition': False,
 'unpersönliche': False,
 'passivkonstruktionen': False,
 'nominalisierungens': False,
 'nebensätze_finale': False,
 'ebensätze_adversative': False,
 'direkte_sprache': False}

In [34]:
# Lernziel 9: Kasus in Adverbialen

example = "in die Schule – in der Schule, sie arbeitet am Abend"
analyze_text(example)

{'präsens': True,
 'präteritum': False,
 'perfekt': False,
 'und_oder': False,
 'kausale_nebensätze': False,
 'nebensätze_dass': False,
 'nebensätze_wenn': False,
 'attribute': True,
 'adverbialen': True,
 'präposition': True,
 'unpersönliche': False,
 'passivkonstruktionen': False,
 'nominalisierungens': False,
 'nebensätze_finale': False,
 'ebensätze_adversative': False,
 'direkte_sprache': False}

In [35]:
# Lernziel 10: Präpositionalergänzungen

example = "Ich glaube an dich"
analyze_text(example)

{'präsens': True,
 'präteritum': False,
 'perfekt': False,
 'und_oder': False,
 'kausale_nebensätze': False,
 'nebensätze_dass': False,
 'nebensätze_wenn': False,
 'attribute': True,
 'adverbialen': True,
 'präposition': True,
 'unpersönliche': False,
 'passivkonstruktionen': False,
 'nominalisierungens': False,
 'nebensätze_finale': False,
 'ebensätze_adversative': False,
 'direkte_sprache': False}

In [36]:
# Lernziel 10: Passivkonstruktionen

example = "Der Tiger wird von der Hyäne gejagt."
analyze_text(example)

{'präsens': True,
 'präteritum': False,
 'perfekt': False,
 'und_oder': False,
 'kausale_nebensätze': False,
 'nebensätze_dass': False,
 'nebensätze_wenn': False,
 'attribute': True,
 'adverbialen': False,
 'präposition': True,
 'unpersönliche': False,
 'passivkonstruktionen': True,
 'nominalisierungens': False,
 'nebensätze_finale': False,
 'ebensätze_adversative': False,
 'direkte_sprache': False}

In [37]:
# Lernziel 11: Unpersönliche Formulierungen

example = "Man nimmt einen Luftballon "
analyze_text(example)

{'präsens': True,
 'präteritum': False,
 'perfekt': False,
 'und_oder': False,
 'kausale_nebensätze': False,
 'nebensätze_dass': False,
 'nebensätze_wenn': False,
 'attribute': True,
 'adverbialen': False,
 'präposition': False,
 'unpersönliche': False,
 'passivkonstruktionen': False,
 'nominalisierungens': False,
 'nebensätze_finale': False,
 'ebensätze_adversative': False,
 'direkte_sprache': False}

In [38]:
# Lernziel 12: Passivkonstruktionen

example = "Der Tiger wird von der Hyäne gejagt."
analyze_text(example)

{'präsens': True,
 'präteritum': False,
 'perfekt': False,
 'und_oder': False,
 'kausale_nebensätze': False,
 'nebensätze_dass': False,
 'nebensätze_wenn': False,
 'attribute': True,
 'adverbialen': False,
 'präposition': True,
 'unpersönliche': False,
 'passivkonstruktionen': True,
 'nominalisierungens': False,
 'nebensätze_finale': False,
 'ebensätze_adversative': False,
 'direkte_sprache': False}

In [39]:
# Lernziel 13: Nominalisierungen

example = "Der Verbrauch in der Mongolei"
analyze_text(example)

{'präsens': False,
 'präteritum': False,
 'perfekt': False,
 'und_oder': False,
 'kausale_nebensätze': False,
 'nebensätze_dass': False,
 'nebensätze_wenn': False,
 'attribute': True,
 'adverbialen': False,
 'präposition': True,
 'unpersönliche': False,
 'passivkonstruktionen': False,
 'nominalisierungens': False,
 'nebensätze_finale': False,
 'ebensätze_adversative': False,
 'direkte_sprache': False}

In [40]:
# Lernziel 14: Finale Nebensätze

example = "Wir schauen das an, damit wir…"
analyze_text(example)

{'präsens': True,
 'präteritum': False,
 'perfekt': False,
 'und_oder': False,
 'kausale_nebensätze': False,
 'nebensätze_dass': False,
 'nebensätze_wenn': False,
 'attribute': True,
 'adverbialen': False,
 'präposition': False,
 'unpersönliche': False,
 'passivkonstruktionen': False,
 'nominalisierungens': False,
 'nebensätze_finale': True,
 'ebensätze_adversative': False,
 'direkte_sprache': False}

In [41]:
# Learning objective 15: Adversative subordinate clauses

example = "Ich finde das gut, obwohl…"
analyze_text(example)

{'präsens': True,
 'präteritum': False,
 'perfekt': False,
 'und_oder': False,
 'kausale_nebensätze': False,
 'nebensätze_dass': False,
 'nebensätze_wenn': False,
 'attribute': True,
 'adverbialen': False,
 'präposition': False,
 'unpersönliche': False,
 'passivkonstruktionen': False,
 'nominalisierungens': False,
 'nebensätze_finale': False,
 'ebensätze_adversative': True,
 'direkte_sprache': False}

In [42]:
# Zusatzlernziel: Wörtliche Rede

example = "Der Mann rief: „Lauf!“ "
analyze_text(example)

{'präsens': False,
 'präteritum': True,
 'perfekt': False,
 'und_oder': False,
 'kausale_nebensätze': False,
 'nebensätze_dass': False,
 'nebensätze_wenn': False,
 'attribute': True,
 'adverbialen': False,
 'präposition': False,
 'unpersönliche': False,
 'passivkonstruktionen': False,
 'nominalisierungens': False,
 'nebensätze_finale': False,
 'ebensätze_adversative': False,
 'direkte_sprache': True}